In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [9]:
df_categories = pd.read_csv('steam_data/categories.csv')
df_games = pd.read_csv('steam_data/games.csv', quotechar='"', escapechar="\\", encoding="utf-8", on_bad_lines="skip")
df_genres = pd.read_csv('steam_data/genres.csv')
df_reviews = pd.read_csv('steam_data/reviews.csv', quotechar='"', escapechar="\\", encoding="utf-8", on_bad_lines="skip")
df_tags = pd.read_csv('steam_data/tags.csv')
df_steamspy_insights = pd.read_csv('steam_data/steamspy_insights.csv', quotechar='"', escapechar="\\", encoding="utf-8", on_bad_lines="skip")

C:\Users\Jonas\AppData\Local\Temp\ipykernel_12080\574882810.py:4: DtypeWarning: Columns (1,3,4,5,9,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_reviews = pd.read_csv('steam_data/reviews.csv', quotechar='"', escapechar="\\", encoding="utf-8", on_bad_lines="skip")


In [10]:
# Pre-processing

,app_id,category
0,10,Family Sharing
1,10,Multi-player
2,10,Online PvP
3,10,PvP
4,10,Shared/Split Screen PvP


Converting json-like object in price_overview column into separate column for easier usage

In [5]:
def extract_price_info(price_str):
    try:
        data = json.loads(price_str)
    except (TypeError, json.JSONDecodeError):
        return pd.Series({
            'final': None,
            'initial': None,
            'currency': None,
            'final_formatted': None,
            'discount_percent': None,
            'initial_formatted': None,
        })
    return pd.Series({
        'final': data.get('final'),
        'initial': data.get('initial'),
        'currency': data.get('currency'),
        'final_formatted': data.get('final_formatted'),
        'discount_percent': data.get('discount_percent'),
        'initial_formatted': data.get('initial_formatted'),
    })

In [6]:
price_details = df_games['price_overview'].apply(extract_price_info)
df_games = pd.concat([df_games, price_details], axis=1)

# to remvoe the original 'price_overview' column
df_games = df_games.drop(columns=['price_overview'])

Cleaning the 'languages' column from <\br> <\strong> and * signs

In [7]:
# <strong> tags
df_games['languages'] = df_games['languages'].str.replace(r'</?strong>', '', regex=True)

# <br> tags (covers <br>, <br/>, or <br />)
df_games['languages'] = df_games['languages'].str.replace(r'</?br\s*/?>', '', regex=True)


# asterisks (*)
df_games['languages'] = df_games['languages'].str.replace('*', '', regex=False)

# to remove space before and the phrase "with full audio support"
df_games['languages'] = df_games['languages'].str.replace(r'\s*with full audio support', '', regex=True)

In [8]:
df_games.to_csv('steam_data/games_cleaned.csv', index=False)

# EDA